# CITEseq Keras Quickstart

This notebook shows how to tune and cross-validate a Keras model for the CITEseq part of the *Multimodal Single-Cell Integration* competition.

It does not show the EDA - see the separate notebook [MSCI EDA which makes sense ⭐️⭐️⭐️⭐️⭐️](https://www.kaggle.com/ambrosm/msci-eda-which-makes-sense).

The CITEseq predictions of the Keras model are then concatenated with the Multiome predictions of @jsmithperera's [Multiome Quickstart w/ Sparse M + tSVD = 32](https://www.kaggle.com/code/jsmithperera/multiome-quickstart-w-sparse-m-tsvd-32) to a complete submission file.

## Summary

The CITEseq part of the competition has sizeable datasets, when compared to the standard 16 GByte RAM of Kaggle notebooks:
- The training input has shape 70988/*22050 (10.6 GByte).
- The training labels have shape 70988/*140.
- The test input has shape 48663/*22050 (4.3 GByte).

Our solution strategy has five elements:
1. **Dimensionality reduction:** To reduce the size of the 10.6 GByte input data, we project the 22050 features to a space with only 64 dimensions by applying a truncated SVD. To these 64 dimensions, we add 144 features whose names shows their importance.
2. **The model:** The model is a sequential dense network with four hidden layers.
3. **The loss function:** The competition is scored by the average Pearson correlation coefficient between the predictions and the ground truth. As this scoring function is differentiable, we can directly use it as loss function for a neural network. This gives neural networks an advantage in comparison to algorithms which use mean squared error as a surrogate loss. 
3. **Hyperparameter tuning with KerasTuner:** We tune the hyperparameters with [KerasTuner](https://keras.io/keras_tuner/). 
4. **Cross-validation:** Submitting unvalidated models and relying only on the public leaderboard is bad practice. The model in this notebook is fully cross-validated with a 3-fold GroupKFold.


In [39]:
import os, gc, pickle, datetime, scipy.sparse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style

from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.preprocessing import StandardScaler, scale, MinMaxScaler
from sklearn.decomposition import TruncatedSVD

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout
from tensorflow.keras.utils import plot_model
# import keras_tuner
# from tensorflow.keras import mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)

DATA_DIR = "C:/Users/Owner/Documents/dev/open-problem/open-problems-multimodal/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

TUNE = False
SUBMIT = True
TRAIN_BASEPATH = "C:/Users/Owner/Documents/dev/open-problem/output/imagedata/multi-minmax/train/"
TEST_BASEPATH = "C:/Users/Owner/Documents/dev/open-problem/output/imagedata/multi-minmax/test/"
USE_SAVED_PCA = True

submission_name = "submission_multiome_image_B0_480.csv"

In [40]:
submission_name = "submission_multiome_image_240.csv"


In [41]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5790987839807812232
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22388146176
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6678153743473512010
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:2b:00.0, compute capability: 8.9"
 xla_global_id: 416903419]

In [42]:
%cd C:/Users/Owner/Documents/dev/open-problem/multiome-image

C:\Users\Owner\Documents\dev\open-problem\multiome-image


A little trick to save time with pip: If the module is already installed (after a restart of the notebook, for instance), pip wastes 10 seconds by checking whether a newer version exists. We can skip this check by testing for the presence of the module in a simple if statement.

In [43]:
%%time
# If you see a warning "Failed to establish a new connection" running this cell,
# go to "Settings" on the right hand side, 
# and turn on internet. Note, you need to be phone verified.
# We need this library to read HDF files.
# if not os.path.exists('/opt/conda/lib/python3.7/site-packages/tables'):
#     !pip install --quiet tables
    

CPU times: total: 0 ns
Wall time: 0 ns


# The scoring function

This competition has a special metric: For every row, it computes the Pearson correlation between y_true and y_pred, and then all these correlation coefficients are averaged. We implement two variants of the metric: The first one is for numpy arrays, the second one for tensors - thanks to @lucasmorin for the [original tensor implementation](https://www.kaggle.com/competitions/open-problems-multimodal/discussion/347595).

In [44]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

def negative_correlation_loss(y_true, y_pred):
    """Negative correlation loss function for Keras
    
    Precondition:
    y_true.mean(axis=1) == 0
    y_true.std(axis=1) == 1
    
    Returns:
    -1 = perfect positive correlation
    1 = totally negative correlation
    """
    my = K.mean(tf.convert_to_tensor(y_pred), axis=1)
    my = tf.tile(tf.expand_dims(my, axis=1), (1, y_true.shape[1]))
    ym = y_pred - my
    r_num = K.sum(tf.multiply(y_true, ym), axis=1)
    r_den = tf.sqrt(K.sum(K.square(ym), axis=1) * float(y_true.shape[-1]))
    r = tf.reduce_mean(r_num / r_den)
    return - r


# Data loading and preprocessing

The metadata is used only for the `GroupKFold`: 

In [45]:
metadata_df = pd.read_csv(FP_CELL_METADATA, index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
metadata_df.shape


(161877, 4)

In [46]:
metadata_df.head()

,day,donor,cell_type,technology
cell_id,,,,
458c2ae2c9b1,2,27678,hidden,multiome
01a0659b0710,2,27678,hidden,multiome
028a8bc3f2ba,2,27678,hidden,multiome
7ec0ca8bb863,2,27678,hidden,multiome
caa0b0022cdc,2,27678,hidden,multiome


In [47]:
cell_index = np.load("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_idxcol.npz", allow_pickle=True)["index"]
meta = metadata_df.reindex(cell_index)
cell_index_test = np.load("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz", allow_pickle=True)["index"]
meta_test = metadata_df.reindex(cell_index_test)

We now define two sets of features:
- `constant_cols` is the set of all features which are constant in the train or test datset. These columns will be discarded immediately after loading.
- `important_cols` is the set of all features whose name matches the name of a target protein. If a gene is named 'ENSG00000114013_CD86', it should be related to a protein named 'CD86'. These features will be used for the model unchanged, that is, they don't undergo dimensionality reduction. 

We read train and test datasets, keep the important columns and convert the rest to sparse matrices.

We apply the truncated SVD to train and test together. The truncated SVD is memory-efficient. We concatenate the SVD output (64 components) with the 144 important features and get the arrays `X` and `Xt`, which will be the input to the Keras model. 

In [48]:
def save(name, model):
    with open(name, 'wb') as f:
        pickle.dump(model, f)

Finally, we read the target array `Y`:

In [49]:
# Read Y
Y = scipy.sparse.load_npz("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/train_multi_targets_values.sparse.npz")

if USE_SAVED_PCA:
    pca2 = pickle.load(open('pca2.pkl', 'rb'))
    train_target = pca2.transform(Y)
else:
    pca2 = TruncatedSVD(n_components=256, random_state=42)
    train_target = pca2.fit_transform(Y)
    print(pca2.explained_variance_ratio_.sum())
    save('pca2.pkl', pca2)

In [50]:
LR_START = 0.01
BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE
SHUFFLE_SIZE = 10000
WIDTH = 480
HEIGHT = 480
EPOCHS = 50
OUTPUT_LEN = train_target.shape[1]

def preprocess_image(image):
#     image = tfio.experimental.image.decode_tiff(path)[...,:3]
    image = tf.io.decode_png(image, channels=1, dtype=tf.dtypes.uint16)
    image = tf.image.resize(image, size=(WIDTH,HEIGHT), method="nearest")
    image = tf.broadcast_to(image, (image.shape[0], image.shape[1], 3))
    image = tf.reshape(image, shape=[WIDTH,HEIGHT,3])

    # image = tf.keras.applications.efficientnet.preprocess_input(image)
    image = tf.keras.applications.efficientnet_v2.preprocess_input(image)

    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

def load_and_preprocess_from_path(path):
    return load_and_preprocess_image(path)

def my_model():
    """Sequential neural network
    
    Returns a compiled instance of tensorflow.keras.models.Model.
    """
    basemodel = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(input_shape=(WIDTH, HEIGHT, 3), weights='imagenet', include_top=False, include_preprocessing=False)
    # basemodel = tf.keras.applications.efficientnet_v2.EfficientNetV2S(input_shape=(WIDTH, HEIGHT, 3), weights='imagenet', include_top=False, include_preprocessing=False)
    # basemodel = tf.keras.applications.efficientnet.EfficientNetB0(input_shape=(WIDTH, HEIGHT, 3), weights='imagenet', include_top=False)

    image_input = tf.keras.layers.Input(shape=(WIDTH,HEIGHT,3))
    out = basemodel(image_input)
    out = tf.keras.layers.GlobalAveragePooling2D()(out)
    out = tf.keras.layers.Dropout(0.5)(out)
    out = tf.keras.layers.Dense(OUTPUT_LEN, activation=None, kernel_regularizer=tf.keras.regularizers.l2(1e-10))(out)

    model = tf.keras.Model(image_input, out)

    # model.compile(optimizer=tf.keras.mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam()), 
    #             loss=root_mean_squared_error,
    #             metrics=[root_mean_squared_error])

    model.compile(optimizer=tf.keras.optimizers.Adam(), 
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()])  
    return model

my_model().summary()



#%%
# Cross-validation
VERBOSE = 2 # set to 2 for more output, set to 0 for less output
N_SPLITS = 3

np.random.seed(1)
tf.random.set_seed(1)

kf = GroupKFold(n_splits=N_SPLITS)

score_list = []
train_paths = TRAIN_BASEPATH + cell_index + ".png"
histories = []

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 480, 480, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 15, 15, 1280)     5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout_20 (Dropout)        (None, 1280)              0         
                                                                 
 dense_20 (Dense)            (None, 256)               327936    
                                                                 
Total params: 6,247,248
Trainable params: 6,186,640
Non-tra

# The model

Our model is a sequential network consisting of a few dense layers. The hyperparameters will be tuned with KerasTuner.

We use the `negative_correlation_loss` defined above as loss function.

In [51]:
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_paths, groups=meta.donor)):
    start_time = datetime.datetime.now()
    model = None
    gc.collect()

    X_tr = train_paths[idx_tr]
    y_tr = train_target[idx_tr]
    X_va = train_paths[idx_va]
    y_va = train_target[idx_va]
    y_va_raw = Y[idx_va]


    train_path_label_ds = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
    train_image_label_ds = train_path_label_ds.map(load_and_preprocess_from_path_label,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(buffer_size=SHUFFLE_SIZE).repeat().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    val_path_label_ds = tf.data.Dataset.from_tensor_slices((X_va, y_va))
    val_image_label_ds = val_path_label_ds.map(load_and_preprocess_from_path_label,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                           patience=3, verbose=VERBOSE)
    es = EarlyStopping(monitor="val_loss",
                       patience=5, 
                       verbose=0,
                       mode="min", 
                       restore_best_weights=True)
    ckpt = tf.keras.callbacks.ModelCheckpoint(f"model/model_{fold}_ckpt", save_best_only=True)
    callbacks = [lr, es, ckpt]
    # callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

    # Construct and compile the model
    model = my_model()

    # Train the model
    history = model.fit(train_image_label_ds, 
                        validation_data=val_image_label_ds, 
                        epochs=EPOCHS,
                        steps_per_epoch=len(X_tr)//BATCH_SIZE,
                        # verbose=VERBOSE,
                        callbacks=callbacks)
    # del X_tr, y_tr
    
    if SUBMIT:
        model.save(f"model/model_{fold}")
    history = history.history
    histories.append(history)
    callbacks, lr = None, None
    
    # We validate the model
    y_va_pred = model.predict(tf.data.Dataset.from_tensor_slices(X_va).map(load_and_preprocess_from_path,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE))
    # corrscore = correlation_score(y_va, y_va_pred)
    corrscore = correlation_score(y_va_raw.todense(), y_va_pred@pca2.components_)

    print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
    del es, X_va, X_tr#, y_va, y_va_pred
    score_list.append(corrscore)

# Show overall score
print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")

#%%
for hist in histories:
    print(hist)


Epoch 1/50
1079/1079 [==============================] - ETA: 0s - loss: 51.3475 - root_mean_squared_error: 7.1657

INFO:tensorflow:Assets written to: model\model_0_ckpt\assets


INFO:tensorflow:Assets written to: model\model_0_ckpt\assets


1079/1079 [==============================] - 519s 467ms/step - loss: 51.3475 - root_mean_squared_error: 7.1657 - val_loss: 40.4572 - val_root_mean_squared_error: 6.3606 - lr: 0.0010
Epoch 2/50
1079/1079 [==============================] - ETA: 0s - loss: 22.2223 - root_mean_squared_error: 4.7141

INFO:tensorflow:Assets written to: model\model_0_ckpt\assets


...

INFO:tensorflow:Assets written to: model\model_2_ckpt\assets


1128/1128 [==============================] - 487s 431ms/step - loss: 15.7346 - root_mean_squared_error: 3.9667 - val_loss: 19.2586 - val_root_mean_squared_error: 4.3885 - lr: 5.0000e-04
Epoch 13/50
1128/1128 [==============================] - 458s 405ms/step - loss: 15.4616 - root_mean_squared_error: 3.9321 - val_loss: 20.6268 - val_root_mean_squared_error: 4.5417 - lr: 5.0000e-04
Epoch 14/50
1128/1128 [==============================] - 457s 406ms/step - loss: 15.1977 - root_mean_squared_error: 3.8984 - val_loss: 19.9452 - val_root_mean_squared_error: 4.4660 - lr: 5.0000e-04
Epoch 15/50
1128/1128 [==============================] - ETA: 0s - loss: 14.9340 - root_mean_squared_error: 3.8645
Epoch 15: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
1128/1128 [==============================] - 457s 405ms/step - loss: 14.9340 - root_mean_squared_error: 3.8645 - val_loss: 20.0829 - val_root_mean_squared_error: 4.4814 - lr: 5.0000e-04
Epoch 16/50
1128/1128 [=================

INFO:tensorflow:Assets written to: model/model_2\assets


INFO:tensorflow:Assets written to: model/model_2\assets


527/527 [==============================] - 47s 88ms/step
Fold 2:  16 epochs, corr =  0.65615
Average  corr = 0.65603
{'loss': [51.34752655029297, 22.2222843170166, 20.243244171142578, 19.14145851135254, 18.297685623168945, 17.61215591430664, 16.957605361938477, 16.488203048706055, 16.022851943969727, 15.576175689697266, 15.222289085388184, 14.872523307800293, 14.548664093017578, 14.258044242858887, 13.985260009765625, 13.249289512634277, 12.941478729248047, 12.718510627746582, 12.555530548095703, 12.242939949035645, 12.073394775390625], 'root_mean_squared_error': [7.165718078613281, 4.714051723480225, 4.499249458312988, 4.375095367431641, 4.277579307556152, 4.196683883666992, 4.117961406707764, 4.0605669021606445, 4.00285530090332, 3.946666717529297, 3.9015750885009766, 3.8564910888671875, 3.8142709732055664, 3.77598237991333, 3.739687204360962, 3.6399574279785156, 3.597426652908325, 3.5663020610809326, 3.5433781147003174, 3.4989912509918213, 3.4746789932250977], 'val_loss': [40.457214

# Prediction and submission

We ensemble the test predictions of all Keras models. 

It has been pointed out in several discussion posts that the first 7476 rows of test (day 2, donor 27678) are identical to the first 7476 rows of train (day 2, donor 32606):
- [CITEseq data: same RNA expression matrices from different donors in day2?](https://www.kaggle.com/competitions/open-problems-multimodal/discussion/349867) (@gwentea)
-[Data contamination between CITEseq train/test datasets?](https://www.kaggle.com/competitions/open-problems-multimodal/discussion/349833) (@aglaros)
- [Leak in public test set](https://www.kaggle.com/competitions/open-problems-multimodal/discussion/349867) (@psilogram)

These rows belong to the public test set; the private leaderboard is not affected. We copy the 7476 rows from the training targets into the test predictions.

At the end we concatenate the CITEseq predictions with @jsmithperera's predictions of the [Multiome Quickstart w/ Sparse M + tSVD = 32](https://www.kaggle.com/code/jsmithperera/multiome-quickstart-w-sparse-m-tsvd-32) notebook to get a complete submission.


In [52]:
def preprocess_image(image):
#     image = tfio.experimental.image.decode_tiff(path)[...,:3]
    image = tf.io.decode_png(image, channels=1, dtype=tf.dtypes.uint16)
    image = tf.image.resize(image, size=(WIDTH,HEIGHT))
    image = tf.broadcast_to(image, (image.shape[0], image.shape[1], 3))
    image = tf.reshape(image, shape=[WIDTH,HEIGHT,3])

    # image = tf.keras.applications.efficientnet.preprocess_input(image)
    image = tf.keras.applications.efficientnet_v2.preprocess_input(image)

    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path(path):
    return load_and_preprocess_image(path)

In [53]:
test_paths = TEST_BASEPATH + cell_index_test + ".png"
preds = np.zeros((len(test_paths), 23418), dtype='float16')
test_path_ds = tf.data.Dataset.from_tensor_slices(test_paths)
test_image_ds = test_path_ds.map(load_and_preprocess_from_path, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

for fold in range(N_SPLITS):
    print(f"Predicting with fold {fold}")

    model = load_model(f"model/model_{fold}")
    preds += (model.predict(test_image_ds)@pca2.components_)/N_SPLITS
    gc.collect()

Predicting with fold 0
874/874 [==============================] - 80s 91ms/step
Predicting with fold 1
874/874 [==============================] - 81s 92ms/step
Predicting with fold 2
874/874 [==============================] - 79s 89ms/step


In [54]:
# n=1
# d = len(test_paths)//n

# preds = np.zeros((len(test_paths), 23418), dtype='float16')
# for i,xx in enumerate(test_paths):
#     for fold in range(N_SPLITS):
#         print(f"Predicting with fold {fold}")
#         train_path_label_ds = tf.data.Dataset.from_tensor_slices(xx)
#         train_image_label_ds = train_path_label_ds.map(load_and_preprocess_from_path, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
#         model = load_model(f"model/model_{fold}")
#         preds[i*d:i*d+d,:] += (model.predict(xx)@pca2.components_)/N_SPLITS
#         gc.collect()
#     print('')
#     del xx
# gc.collect()

In [55]:
# Read the table of rows and columns required for submission
eval_ids = pd.read_parquet("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/evaluation.parquet")
# Convert the string columns to more efficient categorical types
#eval_ids.cell_id = eval_ids.cell_id.apply(lambda s: int(s, base=16))
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

In [56]:
# Prepare an empty series which will be filled with predictions
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

In [57]:
np.save('preds.npy', preds)


In [58]:
%%time
y_columns = np.load("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["columns"]

test_index = np.load("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz",
                    allow_pickle=True)["index"]

CPU times: total: 250 ms
Wall time: 221 ms


In [59]:
cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

In [60]:
eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

In [61]:
submission.iloc[valid_multi_rows] = preds[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

In [62]:
# del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

22

In [63]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86                    NaN
1         c2150f55becb  CD274                   NaN
2         c2150f55becb  CD270                   NaN
3         c2150f55becb  CD155                   NaN
4         c2150f55becb  CD112                   NaN
                                             ...   
65744175  2c53aa67933d  ENSG00000134419    5.566406
65744176  2c53aa67933d  ENSG00000186862    0.036133
65744177  2c53aa67933d  ENSG00000170959    0.044006
65744178  2c53aa67933d  ENSG00000107874    0.948242
65744179  2c53aa67933d  ENSG00000166012    4.929688
Name: target, Length: 65744180, dtype: float32

In [64]:
submission.reset_index(drop=True, inplace=True)
submission.index.name = 'row_id'

In [65]:
cite_submission = pd.read_csv("C:/Users/Owner/Documents/dev/open-problem/citeseq/submission_svd256_wdo.csv")
cite_submission = cite_submission.set_index("row_id")
cite_submission = cite_submission["target"]

In [66]:
submission[submission.isnull()] = cite_submission[submission.isnull()]

In [67]:
submission.isnull().any()

False

In [68]:
submission.to_csv(submission_name)